<a href="https://colab.research.google.com/github/victori4sa/Lab-DEL/blob/main/Sistema_de_gerenciamento_de_despesas_pessoais.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [13]:
# Sistema de gerenciamento de despesas pessoais

import os
import hashlib
import datetime
from collections import defaultdict

# Nome do arquivo que guarda os usuários.
# Formato de cada linha: usuario;senha_hash;salario;estado
# - usuario: nome (string)
# - senha_hash: sha256 da senha (string)
# - salario: string com número formatado (ex: "2500.00") ou vazia se não definido
# - estado: string que guarda a última opção do menu (ex: "2") ou vazia
USERS_FILE = "usuarios.txt"

# Quando gravamos despesas, cada usuário terá seu próprio arquivo:
# exemplo: victori4sa_despesas.txt
# Formato de cada linha de despesas: data;categoria;valor;descricao
def despesas_filename(usuario):
    return "%s_despesas.txt" % (usuario)

def inicializar_usuarios():
    if not os.path.exists(USERS_FILE):
        with open(USERS_FILE, "w", encoding="utf-8") as f:
            pass

def hash_senha(senha):
    return hashlib.sha256(senha.encode("utf-8")).hexdigest()

def carregar_usuarios():
    usuarios = {}
    if not os.path.exists(USERS_FILE):
        return usuarios
    with open(USERS_FILE, "r", encoding="utf-8") as f:
        for linha in f:
            linha = linha.strip()
            if linha == "":
                continue
            partes = linha.split(";")
            if len(partes) < 4:
                continue
            nome = partes[0]
            usuarios[nome] = {
                "senha_hash": partes[1],
                "salario": partes[2],
                "estado": partes[3]
            }
    return usuarios

def salvar_usuarios(usuarios):
    with open(USERS_FILE, "w", encoding="utf-8") as f:
        for nome, info in usuarios.items():
            linha = "%s;%s;%s;%s\n" % (
                nome,
                info.get("senha_hash", ""),
                info.get("salario", ""),
                info.get("estado", "")
            )
            f.write(linha)

def cadastrar():
    usuarios = carregar_usuarios()
    nome = input("Nome de usuário: ").strip()
    if nome == "":
        print("Nome inválido.")
        return
    if nome in usuarios:
        print("Usuário já existe.")
        return
    senha = input("Senha: ").strip()
    if senha == "":
        print("Senha inválida.")
        return
    salario_input = input("Informe seu salário mensal (ou deixe em branco para definir depois): R$ ").strip()
    if salario_input == "":
        salario_para_gravar = ""
    else:
        try:
            salario_val = float(salario_input)
            salario_para_gravar = "%.2f" % (salario_val)
        except Exception:
            print("Valor de salário inválido. Cadastro cancelado.")
            return
    usuarios[nome] = {
        "senha_hash": hash_senha(senha),
        "salario": salario_para_gravar,
        "estado": ""
    }
    salvar_usuarios(usuarios)
    with open(despesas_filename(nome), "w", encoding="utf-8") as f:
        pass
    print("Usuário %s cadastrado com sucesso." % (nome))

def fazer_login():
    usuarios = carregar_usuarios()
    nome = input("Usuário: ").strip()
    senha = input("Senha: ").strip()
    if nome not in usuarios:
        print("Usuário não encontrado.")
        return None, None
    info = usuarios[nome]
    if info.get("senha_hash", "") != hash_senha(senha):
        print("Senha incorreta.")
        return None, None
    print("Login realizado com sucesso!")
    return nome, info

def atualizar_campo_usuario(nome, campo, valor):
    usuarios = carregar_usuarios()
    if nome not in usuarios:
        return
    usuarios[nome][campo] = valor
    salvar_usuarios(usuarios)

def adicionar_despesa(usuario):
    categoria = input("Categoria: ").strip()
    if categoria == "":
        print("Categoria inválida.")
        return
    descricao = input("Descrição (opcional): ").strip()
    valor_str = input("Valor (ex: 12.50): R$ ").strip()
    try:
        valor = float(valor_str)
    except Exception:
        print("Valor inválido. Despesa não registrada.")
        return
    ts = datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S")
    linha = "%s;%s;%.2f;%s\n" % (ts, categoria, valor, descricao)
    arquivo = despesas_filename(usuario)
    with open(arquivo, "a", encoding="utf-8") as f:
        f.write(linha)
    print("Despesa adicionada: %s | %s | R$ %.2f" % (ts, categoria, valor))

def carregar_despesas(usuario):
    arquivo = despesas_filename(usuario)
    despesas = []
    if not os.path.exists(arquivo):
        return despesas
    with open(arquivo, "r", encoding="utf-8") as f:
        for linha in f:
            linha = linha.strip()
            if linha == "":
                continue
            partes = linha.split(";")
            if len(partes) < 4:
                continue
            data = partes[0]
            categoria = partes[1]
            try:
                valor = float(partes[2])
            except Exception:
                valor = 0.0
            descricao = partes[3]
            despesas.append({
                "data": data,
                "categoria": categoria,
                "valor": valor,
                "descricao": descricao
            })
    return despesas

def listar_despesas(usuario):
    despesas = carregar_despesas(usuario)
    if not despesas:
        print("Nenhuma despesa registrada.")
        return
    print("Lista de despesas do usuário %s:" % (usuario))
    for d in despesas:
        print("%s | % -15s | R$ %7.2f | %s" %
              (d["data"], d["categoria"], d["valor"], d["descricao"]))

def total_geral(usuario):
    despesas = carregar_despesas(usuario)
    total = sum(d["valor"] for d in despesas)
    print("Total geral: R$ %7.2f" % (total))
    return total

def total_por_categoria(usuario):
    despesas = carregar_despesas(usuario)
    totais = defaultdict(float)
    for d in despesas:
        totais[d["categoria"]] += d["valor"]
    if not totais:
        print("Nenhuma despesa registrada.")
        return {}
    for cat, val in totais.items():
        print("%s: R$ %7.2f" % (cat, val))
    return dict(totais)

def mostrar_saldo_restante(usuario, info_usuario):
    salario_salvo = info_usuario.get("salario", "")
    if salario_salvo is None or salario_salvo.strip() == "":
        while True:
            s = input("Salário mensal não encontrado. Digite seu salário (ex: 2500.00): R$ ").strip()
            if s == "":
                print("Salário inválido. Digite um valor numérico.")
                continue
            try:
                salario_val = float(s)
                salario_para_gravar = "%.2f" % (salario_val)
                atualizar_campo_usuario(usuario, "salario", salario_para_gravar)
                info_usuario["salario"] = salario_para_gravar
                print("Salário salvo.")
                break
            except Exception:
                print("Valor inválido. Digite um número (ex: 2500.00).")
    try:
        salario_float = float(info_usuario.get("salario", "0") or 0.0)
    except Exception:
        salario_float = 0.0
    total = total_geral(usuario)
    saldo = salario_float - total
    print("Salário: R$ %7.2f" % (salario_float))
    print("Saldo restante: R$ %7.2f" % (saldo))
    return saldo

def gerar_relatorio(usuario, info_usuario):
    despesas = carregar_despesas(usuario)
    salario_salvo = info_usuario.get("salario", "")
    if salario_salvo is None or salario_salvo.strip() == "":
        while True:
            s = input("Digite seu salário para calcular o saldo (ex: 2500.00): R$ ").strip()
            try:
                salario_val = float(s)
                salario_salvo = "%.2f" % (salario_val)
                atualizar_campo_usuario(usuario, "salario", salario_salvo)
                print("Salário salvo.")
                break
            except Exception:
                print("Valor inválido. Tente novamente.")
    try:
        salario_float = float(salario_salvo)
    except Exception:
        salario_float = 0.0
    totais_cat = defaultdict(float)
    for d in despesas:
        totais_cat[d["categoria"]] += d["valor"]
    total = sum(d["valor"] for d in despesas)
    saldo = salario_float - total
    agora = datetime.datetime.now().strftime("%H-%M-%S_%d-%m-%Y")
    nome_arquivo = "relatorio_%s_%s.txt" % (usuario, agora)
    with open(nome_arquivo, "w", encoding="utf-8") as f:
        f.write("RELATÓRIO DE DESPESAS - Usuário: %s\n" % (usuario))
        f.write("Gerado em: %s\n\n" % (agora.replace("_", " ")))
        f.write("Lista de despesas:\n")
        if not despesas:
            f.write("(nenhuma despesa)\n")
        else:
            for d in despesas:
                f.write("%s | %s | R$ %.2f | %s\n" %
                        (d["data"], d["categoria"], d["valor"], d["descricao"]))
        f.write("\nTotais por categoria:\n")
        if not totais_cat:
            f.write("(nenhuma categoria)\n")
        else:
            for cat, val in totais_cat.items():
                f.write("%s: R$ %.2f\n" % (cat, val))
        f.write("\nTotal geral: R$ %.2f\n" % (total))
        f.write("Salário informado: R$ %.2f\n" % (salario_float))
        f.write("Saldo restante: R$ %.2f\n" % (saldo))
    print("Relatório gerado: %s" % (nome_arquivo))

def comparar_periodos(usuario):
    despesas = carregar_despesas(usuario)
    if not despesas:
        print("Nenhuma despesa registrada para comparar.")
        return
    def ler_data(prompt):
        while True:
            s = input(prompt).strip()
            try:
                return datetime.datetime.strptime(s, "%Y-%m-%d").date()
            except Exception:
                print("Data inválida. Use o formato YYYY-MM-DD.")
    print("Período 1:")
    p1i = ler_data("  Início (YYYY-MM-DD): ")
    p1f = ler_data("  Fim   (YYYY-MM-DD): ")
    print("Período 2:")
    p2i = ler_data("  Início (YYYY-MM-DD): ")
    p2f = ler_data("  Fim   (YYYY-MM-DD): ")
    def filtrar(p_inicio, p_fim):
        res = []
        for d in despesas:
            try:
                data_obj = datetime.datetime.strptime(d["data"], "%Y-%m-%d %H:%M:%S").date()
            except Exception:
                continue
            if p_inicio <= data_obj <= p_fim:
                res.append(d)
        return res
    total1 = sum(x["valor"] for x in filtrar(p1i, p1f))
    total2 = sum(x["valor"] for x in filtrar(p2i, p2f))
    print("Total período 1: R$ %.2f" % (total1))
    print("Total período 2: R$ %.2f" % (total2))
    if total1 > total2:
        print("Você gastou MAIS no período 1.")
    elif total2 > total1:
        print("Você gastou MAIS no período 2.")
    else:
        print("Os períodos têm o mesmo valor gasto.")

# Função: permitir que o usuário altere manualmente seu salário
def alterar_salario(usuario, info_usuario):
    while True:
        novo = input("Digite o novo salário (ex: 2500.00): R$ ").strip()
        try:
            salario_val = float(novo)
            salario_formatado = "%.2f" % salario_val
            atualizar_campo_usuario(usuario, "salario", salario_formatado)
            info_usuario["salario"] = salario_formatado
            print("Salário atualizado com sucesso!")
            return
        except Exception:
            print("Valor inválido. Tente novamente.")

def menu_inicial():
    while True:
        print("Opções:")
        print("1 - Cadastrar usuário")
        print("2 - Fazer login")
        print("3 - Sair")
        escolha = input("O que deseja? ").strip()
        if escolha in ("1", "2", "3"):
            return escolha
        else:
            print("Opção inválida. Digite 1, 2 ou 3.")

def menu_principal(usuario, info_usuario):
    estado_salvo = info_usuario.get("estado", "") or ""
    if estado_salvo != "":
        print("Você saiu da última vez enquanto estava na opção: %s" % (estado_salvo))
        resp = input("Deseja continuar dessa opção? (s/n): ").strip().lower()
        if resp == "s":
            cmd = estado_salvo
        else:
            cmd = ""
    else:
        cmd = ""
    while True:
        print("Opções:")
        print("1 - Adicionar despesa")
        print("2 - Listar despesas")
        print("3 - Total geral")
        print("4 - Total por categoria")
        print("5 - Saldo restante do mês")
        print("6 - Gerar relatório")
        print("7 - Selecionar período e comparar")
        print("8 - Alterar salário")
        print("9 - Sair")
        if cmd == "":
            cmd = input("O que deseja? ").strip()
        atualizar_campo_usuario(usuario, "estado", cmd)
        if cmd == "1":
            adicionar_despesa(usuario)
        elif cmd == "2":
            listar_despesas(usuario)
        elif cmd == "3":
            total_geral(usuario)
        elif cmd == "4":
            total_por_categoria(usuario)
        elif cmd == "5":
            mostrar_saldo_restante(usuario, info_usuario)
        elif cmd == "6":
            gerar_relatorio(usuario, info_usuario)
        elif cmd == "7":
            comparar_periodos(usuario)
        elif cmd == "8":
            alterar_salario(usuario, info_usuario)
        elif cmd == "9":
            atualizar_campo_usuario(usuario, "estado", "")
            print("Saindo...")
            break
        else:
            print("Opção inválida. Digite um número de 1 a 9.")
        cmd = ""

def main():
    inicializar_usuarios()
    while True:
        escolha = menu_inicial()
        if escolha == "1":
            cadastrar()
        elif escolha == "2":
            usuario, info = fazer_login()
            if usuario is not None:
                menu_principal(usuario, info)
                resp = input("Deseja encerrar o programa? (s/n): ").strip().lower()
                if resp == "s":
                    print("Programa encerrado. Até mais!")
                    break
        elif escolha == "3":
            print("Encerrando. Até mais!")
            break

main()

Opções:
1 - Cadastrar usuário
2 - Fazer login
3 - Sair
O que deseja? 3
Encerrando. Até mais!
